In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [5]:
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm.auto import tqdm
tqdm.pandas()
%matplotlib inline

RANDOM_SEED = 42

# Import Data

In [4]:
df = pd.read_csv("fraud_detection_train.csv")
df.head()

,visit_id,kdkc,dati2,typeppk,jkpst,umur,jnspelsep,los,cmg,severitylevel,...,proc63_67,proc68_70,proc71_73,proc74_75,proc76_77,proc78_79,proc80_99,proce00_e99,procv00_v89,label
0,1,1107,150,SB,P,64,2,0,F,0,...,0,0,0,0,0,0,0,0,0,1
1,2,1303,200,C,L,45,1,9,E,3,...,0,0,0,0,0,0,4,0,0,1
2,3,1114,172,B,P,34,2,0,Q,0,...,0,0,0,0,0,0,0,0,0,1
3,4,601,90,SC,L,34,2,0,Q,0,...,0,0,0,0,0,0,0,0,0,1
4,5,1006,130,B,L,27,2,0,F,0,...,0,0,0,0,0,0,0,0,0,1


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200217 entries, 0 to 200216
Data columns (total 53 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   visit_id       200217 non-null  int64 
 1   kdkc           200217 non-null  int64 
 2   dati2          200217 non-null  int64 
 3   typeppk        200217 non-null  object
 4   jkpst          200217 non-null  object
 5   umur           200217 non-null  int64 
 6   jnspelsep      200217 non-null  int64 
 7   los            200217 non-null  int64 
 8   cmg            200217 non-null  object
 9   severitylevel  200217 non-null  int64 
 10  diagprimer     200217 non-null  object
 11  dx2_a00_b99    200217 non-null  int64 
 12  dx2_c00_d48    200217 non-null  int64 
 13  dx2_d50_d89    200217 non-null  int64 
 14  dx2_e00_e90    200217 non-null  int64 
 15  dx2_f00_f99    200217 non-null  int64 
 16  dx2_g00_g99    200217 non-null  int64 
 17  dx2_h00_h59    200217 non-null  int64 
 18  dx2_

In [7]:
df.nunique()

visit_id         200217
kdkc                126
dati2               486
typeppk              25
jkpst                 2
umur                105
jnspelsep             2
los                 142
cmg                  23
severitylevel         4
diagprimer           21
dx2_a00_b99           5
dx2_c00_d48           4
dx2_d50_d89           4
dx2_e00_e90           7
dx2_f00_f99           3
dx2_g00_g99           5
dx2_h00_h59           5
dx2_h60_h95           4
dx2_i00_i99           7
dx2_j00_j99           5
dx2_koo_k93           1
dx2_l00_l99           4
dx2_m00_m99           4
dx2_n00_n99           5
dx2_o00_o99           7
dx2_p00_p96          14
dx2_q00_q99           7
dx2_r00_r99           5
dx2_s00_t98           8
dx2_u00_u99           1
dx2_v01_y98           3
dx2_z00_z99           6
proc00_13             5
proc14_23             6
proc24_27             4
proc28_28             3
proc29_31             3
proc_32_38            6
proc39_45             5
proc46_51             4
proc52_57       

# EDA

## Quick peak

In [8]:
df.corr()["label"]

ValueError: could not convert string to float: 'SB'

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
sns.distplot(df.umur, ax=ax1)
sns.distplot(df.los, ax=ax2)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
sns.distplot(df.jnspelsep, ax=ax1)
sns.distplot(df.severitylevel, ax=ax2)

In [ ]:
imp = []
for col in df.columns.tolist():
    nunique = df[col].nunique()
    print(f"{col} : {nunique}")
    if nunique > 1 and nunique != df.shape[0]:
        imp.append(col)

In [ ]:
df2 = df[imp].copy()
df2["los_log"] = np.log(df2.los + 2)

In [ ]:
sns.distplot(df2.los_log)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
sns.distplot(df2[df2.los_log > 3].los_log, ax=ax1)
sns.distplot(df2[df2.los_log < 3].los_log, ax=ax2)

In [ ]:
df2.drop(df2[(df2.los > 40)].index, inplace=True)
df2.shape

In [ ]:
df2[(df2.umur > 90)].label.value_counts()

In [ ]:
sns.distplot(df2[(df2.umur < 90)].umur)

In [ ]:
df2[(df2.los==0) & (df2.jnspelsep == 1) ].label.value_counts()

In [ ]:
# df2.drop(df2[(df2.los==0) & (df2.label==0) & (df2.jnspelsep == 1)].index, inplace=True)
# df2.shape

# Feature Engineering

In [ ]:
proc_cols = [col for col in df2.columns if "proc" in col]
proc_cols

In [ ]:
dx2_cols = [col for col in df2.columns if "dx2" in col]
dx2_cols

In [ ]:
def count_dx2(row):
    count = 0
    for col in dx2_cols:
        count += 1 if row[col] > 0 else 0

    row["count_dx2"] = count
    return row

def count_proc(row):
    count = 0
    for col in proc_cols:
        count += 1 if row[col] > 0 else 0

    row["count_proc"] = count
    return row

In [ ]:
# 1) Masa Balita: 0–5 Tahun;
# 2) Masa Kanak-Kanak: 5–11 Tahun;
# 3) Masa Remaja Awal: 12–16 Tahun; 
# 4) Masa Remaja Akhir: 17–25 Tahun;
# 5) Masa Dewasa Awal: 26–35 Tahun;
# 6) Masa Dewasa Akhir: 36–45 Tahun;
# 7) Masa Lansia Awal: 46–55 Tahun; 
# 8) Masa Lansia Akhir: 56–65 Tahun; dan 
# 9) Masa Manula: > 65 Tahun.
    
def encode_umur(x):
    if x.umur <= 5:
        x['encode_umur'] = 1
    elif x.umur <= 11:
        x['encode_umur'] = 2
    elif x.umur <= 16:
        x['encode_umur'] = 3
    elif x.umur <= 25:
        x['encode_umur'] = 4
    elif x.umur <= 35:
        x['encode_umur'] = 5
    elif x.umur <= 45:
        x['encode_umur'] = 6
    elif x.umur <= 55:
        x['encode_umur'] = 7
    elif x.umur <= 65:
        x['encode_umur'] = 8
    else:
        x['encode_umur'] = 9

    return x


In [ ]:
def concat_dati2_kdkc(row):
    row["dati2_kdkc"] = str(row["dati2"]) + "_" + str(row["kdkc"])
    return row

In [ ]:
def is_baby(row):
    row["is_baby"] = 1 if row.umur == 0 else 0
    return row

In [ ]:
def apply_row(row):
    row = concat_dati2_kdkc(row)
#     row = is_baby(row)
    return row

In [ ]:
tempdf = df2.apply(lambda x: apply_row(x), axis=1)

In [ ]:
df2 = tempdf.copy()

In [ ]:
df2[df2.label==1].dati2_kdkc.value_counts()

In [ ]:
df2["count_proc"] = 0
for col in proc_cols:
    df2["count_proc"] += df2[col]

In [ ]:
df2["count_dx2"] = 0
for col in dx2_cols:
    df2["count_dx2"] += df2[col]

In [ ]:
from sklearn.model_selection import cross_val_score, RepeatedStratifiedKFold, train_test_split, cross_validate

In [ ]:
drop_cols = ["los", "jnspelsep", "jkpst", "kdkc"]
astype_dict = {"dati2": "object", "severitylevel": "object"}

In [ ]:
X = df2.drop(["label"] + drop_cols + proc_cols + dx2_cols, axis=1)
y = df2.label.copy()

X = X.astype(astype_dict)
# X = pd.get_dummies(X)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.9, stratify=y, random_state=42)

In [ ]:
# X["proc_dx2"] = X.count_proc - X.count_dx2
# X = X.drop(["count_proc", "count_dx2"], axis=1)

In [ ]:
X.info()

In [ ]:
from sklearn.metrics import classification_report

# Model Predict

In [ ]:
# from xgboost import XGBClassifier

# param = {
#     'bootstrap': True,
#     'max_depth': 50,
#     'max_features': 6,
#     'min_samples_leaf': 4,
#     'min_samples_split': 12
# }

# xgb = XGBClassifier(random_state=42, **param)
# scoring = ['precision', 'recall', 'f1', 'accuracy']
# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42)
# score = cross_validate(xgb, X, y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score=False)

# print(score)

In [ ]:
from catboost import CatBoostClassifier

cat_features = [index for index, column in enumerate(X.columns) if X[column].dtype == "object"]
params = {
    "loss_function": "CrossEntropy",
    "eval_metric": "Precision",
    "random_seed": 42,
    "task_type": "CPU",
    "cat_features": cat_features,
    "early_stopping_rounds": 200,
    "iterations": 2000,
    "verbose": 100,
    "max_depth": 10,
}

cat = CatBoostClassifier(**params)
scoring = ['precision', 'recall', 'f1', 'accuracy']
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42)
score = cross_validate(cat, X, y, scoring=scoring, cv=cv, n_jobs=-1, return_train_score=False)

print(score)

In [ ]:
avg_precision = score["test_precision"].mean()
avg_recall = score["test_recall"].mean()
avg_f1 = score["test_f1"].mean()
avg_accuracy = score["test_accuracy"].mean()
print("avg_precision", score["test_precision"].mean())
print("avg_recall", score["test_recall"].mean())
print("avg_f1", score["test_f1"].mean())
print("final_score", 0.4*avg_precision + 0.3*avg_recall + 0.3*avg_accuracy)

In [ ]:
import collections
from sklearn.model_selection import train_test_split

TRAIN_SIZE = 0.8
VAL_SIZE = 0.1
TEST_SIZE = 0.1

def train_val_test_split(X, y, train_size):
    """Split dataset into data splits."""
    X_train, X_, y_train, y_ = train_test_split(X, y, train_size=TRAIN_SIZE, stratify=y, random_state=42)
    X_val, X_test, y_val, y_test = train_test_split(X_, y_, train_size=0.5, stratify=y_, random_state=42)
    return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = train_val_test_split(
    X=X, y=y, train_size=TRAIN_SIZE)
print (f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print (f"X_val: {X_val.shape}, y_val: {y_val.shape}")
print (f"X_test: {X_test.shape}, y_test: {y_test.shape}")

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.8, stratify=y, random_state=42)

In [ ]:
from catboost import CatBoostClassifier

cat_features = [index for index, column in enumerate(X_train.columns) if X_train[column].dtype == "object"]
params = {
    "loss_function": "CrossEntropy",
    "eval_metric": "Precision",
    "random_seed": 42,
    "task_type": "CPU",
    "cat_features": cat_features,
    "early_stopping_rounds": 200,
    "iterations": 2000,
    "verbose": 100,
    "max_depth": 8,
}

cat = CatBoostClassifier(**params)

In [ ]:
cat.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, plot=True)

In [ ]:
y_pred = cat.predict(data=X_test)
print(classification_report(y_test, y_pred))

In [ ]:
for i, x in enumerate(list(X_train.columns)):
    print(x, cat.get_feature_importance()[i])

In [ ]:
#@title Feature Importances
import matplotlib.pyplot as plt
features = list(X_train.columns)
importances = cat.feature_importances_
indices = np.argsort(importances)

plt.title('Feature Importances')
plt.barh(range(len(indices)), importances[indices], color='b', align='center')
plt.yticks(range(len(indices)), [features[i] for i in indices])
plt.xlabel('Relative Importance')
plt.show()

# Apply to Test Data

In [ ]:
import pandas as pd

In [ ]:
df_test = pd.read_csv("../input/bpjshackathon2021/pacilgo.ArtificalIntelligence.FraudDetectVal/Fraud Detection/fraud_detection_val.csv")

In [ ]:
df_test[(df_test.umur > 90)].shape

In [ ]:
new_imp = [x for x in imp if x!="label"]

In [ ]:
test = df_test[new_imp].copy()
test["los_log"] = np.log(test.los + 2)

In [ ]:
test2 = test.apply(lambda x: apply_row(x), axis=1)

In [ ]:
test2["count_proc"] = 0
for col in proc_cols:
    test2["count_proc"] += test2[col]

In [ ]:
test2["count_dx2"] = 0
for col in dx2_cols:
    test2["count_dx2"] += test2[col]

In [ ]:
test3 = test2.drop( drop_cols + proc_cols + dx2_cols, axis=1)

In [ ]:
test3 = test3.astype(astype_dict)

In [ ]:
y_pred = cat.predict(data=test3)

In [ ]:
sub = pd.DataFrame({"visit_id": df_test.visit_id, "predict_label": y_pred})
sub.head()

In [ ]:
sub.to_csv('submission.csv', index=False)

# Test 2 for data train 0.9

In [ ]:
# X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.9, stratify=y, random_state=42)

In [ ]:
# from catboost import CatBoostClassifier

# cat_features = [index for index, column in enumerate(X_train.columns) if X_train[column].dtype == "object"]
# params = {
#     "loss_function": "CrossEntropy",
#     "eval_metric": "Precision",
#     "random_seed": 42,
#     "task_type": "CPU",
#     "cat_features": cat_features,
#     "early_stopping_rounds": 200,
#     "iterations": 2000,
#     "verbose": 100,
# }

# cat = CatBoostClassifier(**params)

In [ ]:
# cat.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True, plot=True)

In [ ]:
# y_pred = cat.predict(data=X_val)
# print(classification_report(y_val, y_pred))

In [ ]:
# X = df2.drop(["label", "los"] + proc_cols + dx2_cols, axis=1)
# y = df2.label.copy()

# # X = pd.get_dummies(X)

In [ ]:
# X = X.astype({"kdkc": "object", "dati2": "object", "encode_umur": "object", "severitylevel": "object"})

In [ ]:
# from catboost import CatBoostClassifier

# cat_features = [index for index, column in enumerate(X.columns) if X[column].dtype == "object"]
# params = {
#     "loss_function": "CrossEntropy",
#     "eval_metric": "F1",
#     "random_seed": 42,
#     "task_type": "CPU",
#     "cat_features": cat_features,
#     "early_stopping_rounds": 200,
#     "iterations": 1000,
#     "verbose": 100,
# }

# cat = CatBoostClassifier(**params)

# cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=2, random_state=42)
# score = cross_val_score(cat, X, y, scoring="precision", cv=cv, n_jobs=-1)

# print(score)

## Preprocessing Data

In [ ]:
# Mengubah kategori menjadi numerik
from sklearn.preprocessing import LabelEncoder

categorical_cols = ['typeppk', 'jkpst', 'cmg', 'diagprimer']
encoder = LabelEncoder()

for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])

# Cek data setelah transformasi
df.head()

## Split Data untuk Training dan Testing

In [ ]:
from sklearn.model_selection import train_test_split

X = df.drop(columns=['visit_id', 'label'])  # Drop ID yang tidak relevan
y = df['label']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Cek distribusi data
y_train.value_counts(normalize=True), y_test.value_counts(normalize=True)

## Training Model Machine Learning

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

# Model Random Forest
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Prediksi
y_pred = model.predict(X_test)

# Evaluasi
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))